In [1]:
### Importing packages
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize
%matplotlib inline

In [2]:
### Random seed generation
#     a_list:  values of a in the ax^2 form
def random_seed(a_list):
    sigs = np.sqrt(1.0 / (2.0 * np.array(a_list)))
    return np.random.normal(0.0, sigs)

In [3]:
### Update function to generate new vectors
#     prev_list:    previous values of variables
#     factor:       how much we shift variables by
#     a_list:       values of a in ax^2 form
def update(prev_list, factor, a_list):
    assert (len(prev_list) == len(a_list)),"prev_list and a_list must be of the same length"
    sigs = np.sqrt(1.0 / (2.0 * np.array(a_list)))
    diffs = factor * np.random.normal(0.0, sigs)
    return np.array(prev_list) + diffs

In [4]:
### Writing probability function for generic coupled system
#     x_list:   value of variables (as a list of two variables)
#     a_list:   values of a in ax^2 form
#     b_list:   values of b
def make_potential(x_list, a_list, b_list, pairs):
    assert(len(x_list) == len(a_list)),"x_list and a_list must be of the same length"
    assert(len(b_list) == len(pairs)),"a_list and b_list lengths aren't compatible"
    var_diffs = np.array(a_list) * (np.array(x_list) ** 2)
    inter_dists = [(x_list[i] - x_list[j]) ** 2 for (i,j) in pairs]
    if not len(inter_dists):
        inter_vars = 0.0
    else:
        inter_vars = np.array(b_list) * np.array(inter_dists)
    
    energy = np.sum(var_diffs) + np.sum(inter_vars)
    probability = np.exp(-1 * energy)
    return probability

In [5]:
### Somewhat generalized Metropolis algorithm
#     seed_generator(a_list): function that generates starting seed for a given vector of a values (ax^2)
#     update(prev_list, factor, a_list): function that takes a point and returns an updated point for consideration
#     prob(x_list, a_list, b_list): computes the probability of a point x_list given a set of values a_list and b_list
#     traj_len: length of trajectory
#     factor: value of factor for use in update function
#     a_list: values of a in ax^2 form
#     b_list: values of b
def mod_metropolis(seed_generator, update, prob, traj_len, factor, a_list, b_list):
    traj = []
    point = seed_generator(a_list)
    traj.append(point)
    while (len(traj) < traj_len):
        new_point = update(point, factor, a_list)
        acceptance_ratio = prob(new_point, a_list, b_list) / prob(point, a_list, b_list)
        u = np.random.uniform()
        if u <= acceptance_ratio:
            point = new_point
            traj.append(point)
        else:
            traj.append(point)
    return traj

In [12]:
### Function definitions
def compute_partition(a_1, a_2, a_3, b_2, b_3):
    numer = np.pi ** 1.5
    denom = np.sqrt(a_1*a_2*a_3
                   +a_1*a_3*b_2
                   +a_3*b_2*b_3
                   +a_1*a_2*b_3
                   +a_1*a_3*b_3
                   +a_1*b_2*b_3
                   +a_2*a_3*b_2
                   +a_2*b_2*b_3)
    return numer / denom

def compute_prob(point, theta):
    
    a_1 = theta[0]
    a_2 = theta[1]
    a_3 = theta[2]
    b_2 = theta[3]
    b_3 = theta[4]
    
    x_1 = point[0]
    x_2 = point[1]
    x_3 = point[2]
    
    likelihood = np.exp(-a_1 * (x_1 ** 2) - a_2 * (x_2 ** 2) - a_3 * (x_3 ** 2)
                        -b_2 * (x_1 - x_2) ** 2 - b_3 * (x_2 - x_3) ** 2)
    return likelihood / compute_partition(a_1, a_2, a_3, b_2, b_3)

def compute_loss(theta, datapoints):
    probs = np.array([compute_prob(point, theta) for point in datapoints])
    logs  = np.log(probs)
    return -1.0 * np.sum(logs)

for trial in range(10):
    ### Generating trajectory/datapoints
    pairs = [(0,1), (1,2)]
    traj_len = int(1e5)
    a_list = np.random.uniform(0.5, 5.0, 3)
    b_list = np.random.uniform(0.5, 5.0, 2)
    num_vars = len(a_list)

    potential = lambda point, a_list, b_list : make_potential(point, a_list, b_list, pairs)
    datapoints = mod_metropolis(random_seed, update, potential, traj_len, 0.1, a_list, b_list)

    ### Parameter estimation ???
    bnds = ((0.01, 10.0), (0.01, 10.0), (0.01, 10.0), (0.01, 10.0), (0.01, 10.0))
    res = minimize(compute_loss, x0=[1.0, 1.0, 1.0, 1.0, 1.0], args=datapoints, method='SLSQP', bounds=bnds)
    print("Trial " + str(trial + 1) + ":")
    print("Real Parameters:  " + str(np.append(a_list, b_list)))
    print("Computed Results: " + str(res.x))
    print()

Trial 1:
Real Parameters:  [2.58597853 0.63656001 3.45358626 2.46428722 3.84567742]
Computed Results: [2.45024184 0.534803   3.2461775  2.56664158 3.92494208]

Trial 2:
Real Parameters:  [2.88652755 4.12640624 4.78598693 1.87049581 4.43121902]
Computed Results: [3.14844875 4.38019578 4.68206943 2.0166658  4.64687047]

Trial 3:
Real Parameters:  [1.81207695 2.10748647 2.60573525 1.28859031 4.26902862]
Computed Results: [1.80698967 2.38999496 2.68345245 1.21839921 4.24423349]

Trial 4:
Real Parameters:  [4.90342378 0.83249503 3.89336648 3.62575399 1.20980599]
Computed Results: [4.42768918 1.04583784 3.84874575 3.78990007 0.92893349]

Trial 5:
Real Parameters:  [1.37228321 1.437583   1.92745344 3.00079785 1.50721719]
Computed Results: [1.25796559 1.27344741 1.91160764 3.07448772 1.6325279 ]

Trial 6:
Real Parameters:  [1.05332199 4.84026549 3.63341248 3.46242683 2.17346436]
Computed Results: [1.25341562 4.77616893 3.49756788 3.27062496 2.05470585]

Trial 7:
Real Parameters:  [2.21839509 1

In [ ]:
### What next
Assumptions:
1. Graph structure
2. Parametric Form (exp(-a_1*x_1^2 - ... -b_2*(x_2-x_1)^2 - ...))
3. Directed model